In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
print(y.shape, tX.shape)

(250000,) (250000, 30)


In [3]:
with open(DATA_TRAIN_PATH, "r") as f:
    headers = f.readline().strip().split(',')[2:]

## Do your thing crazy machine learning thing here :) ...

In [4]:
from implementations import *

In [5]:
xs, ids = process_data(tX)
assert len(xs) == len(ids)

ys = y[ids[0]], y[ids[1]], y[ids[2]]

s = 0
for i in range(len(xs)):
    s += xs[i].shape[0]
    print(xs[i].shape, ys[i].shape)

assert s == tX.shape[0]


(99913, 14) (99913, 1)
(77544, 17) (77544, 1)
(72543, 25) (72543, 1)


In [6]:
# train model
degrees = np.arange(1,15)
lambdas = np.logspace(-6, -2, 50)
threeModels = ThreeModels(xs,ys)
threeModels.train(degrees, lambdas, 5)

Start training for model model0
Start training for model model1
Start training for model model2
model1
10.14% of training done
Intermediate result:
Degree:  2 Lambda: 0.0000054287 Accuracy: 0.790
model0
10.14% of training done
Intermediate result:
Degree:  2 Lambda: 0.0000115140 Accuracy: 0.840
model2
10.14% of training done
Intermediate result:
Degree:  2 Lambda: 0.0000294705 Accuracy: 0.812
model1
20.14% of training done
Intermediate result:
Degree:  3 Lambda: 0.0000625055 Accuracy: 0.791
model0
20.14% of training done
Intermediate result:
Degree:  3 Lambda: 0.0000167683 Accuracy: 0.840
model2
20.14% of training done
Intermediate result:
Degree:  3 Lambda: 0.0008685114 Accuracy: 0.814
model1
30.14% of training done
Intermediate result:
Degree:  5 Lambda: 0.0000037276 Accuracy: 0.794
model0
30.14% of training done
Intermediate result:
Degree:  5 Lambda: 0.0000010000 Accuracy: 0.841
model2
30.14% of training done
Intermediate result:
Degree:  5 Lambda: 0.0000021210 Accuracy: 0.815
mode

In [9]:
threeModels.save_CV_results("../results/cv_res_trigo")

In [9]:
threeModels2 = ThreeModels(xs,ys)
threeModels2.load_CV_results("../results/cv_res")
print((threeModels.models[0].accuracy == threeModels2.models[0].accuracy).all())

True


In [10]:
threeModelsHigher = ThreeModels(xs,ys)
threeModelsHigher.train(np.array([14]), lambdas, 5)

Start training for model model0
Start training for model model1
Start training for model model2
model1
11.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000000254 Accuracy: 0.789
model0
11.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000000305 Accuracy: 0.768
model2
11.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000000643 Accuracy: 0.802
model1
21.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000000933 Accuracy: 0.794
model0
21.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000004132 Accuracy: 0.808
model1
31.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000018307 Accuracy: 0.801
model0
31.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000015199 Accuracy: 0.821
model2
21.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000004132 Accuracy: 0.818
model1
41.00% of training done
Intermediate result:
Degree: 14 Lambda: 0.0000018307 Accuracy: 0.801
mode

In [11]:
threeModels.save_CV_results("../results/cv_res_14")

In [7]:
for m in threeModels.models:
    print(m.name)
    print(m.best_deg)
    print(m.best_lambda)

NameError: name 'threeModels' is not defined

## Generate predictions and save ouput in csv format for submission:

In [8]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
print(ids_test)

[350000 350001 350002 ... 918235 918236 918237]


In [10]:
model_params = [(12, 0.00010974987654930568), (12,0.002595024211399737), (12, 0.001023531021899027)]
predModel = ThreeModels(xs, ys)

for i,m in enumerate(predModel.models):
    m.best_deg = threeModels.models[i].best_deg
    m.best_lambda = threeModels.models[i].best_lambda

predModel.compute_ws()

xs_test, indexes_per_cat = process_data(tX_test)
y_pred = np.zeros(len(tX_test))

for i, model in enumerate(predModel.models):
    x_poly = build_poly(xs_test[i], model.best_deg)
    y_pred[indexes_per_cat[i]] = predict_labels(model.w, x_poly)




In [11]:
OUTPUT_PATH = '../results/prediction.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)